In [1]:
import os
os.chdir('/Volumes/Projects/autopilotml')

In [62]:
from autopilotml.load_data import read_csv
from autopilotml.preprocessing import imputation, drop_missing_rows, drop_missing_columns, inter_quartile_range, remove_outliers_zscore
from autopilotml.transformation import onehot_transform, label_transform, ordinal_transform
from autopilotml.scaling import standard_scale, target_scale, maxabs_scale, robust_scale
from autopilotml.feature_selection import rfe
from datetime import datetime
import mlflow
from mlflow.tracking import MlflowClient
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR, SVC

# Data Loading

In [88]:
df = read_csv("/Volumes/Projects/autopilotml/autopilotml/dataset/titanic_train.csv")
df.head()

PassengerId  Survived  Pclass  \
0            1       0.0       3   
1            2       1.0       1   
2            3       1.0       3   
3            4       1.0       1   
4            5       NaN       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     888 non-null    float64
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(3), int64(4), object(5)
memory usage: 83.7+ KB


# Data Preprocessing

In [5]:
# Drop high cardinality features
df = df.drop(['PassengerId'], axis=1)

In [6]:
df.isnull().sum()

Survived      3
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [7]:
df = df.dropna(subset=['Survived'], axis = 0)
df

Survived  Pclass                                               Name  \
0         0.0       3                            Braund, Mr. Owen Harris   
1         1.0       1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2         1.0       3                             Heikkinen, Miss. Laina   
3         1.0       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
7         0.0       3                     Palsson, Master. Gosta Leonard   
..        ...     ...                                                ...   
886       0.0       2                              Montvila, Rev. Juozas   
887       1.0       1                       Graham, Miss. Margaret Edith   
888       0.0       3           Johnston, Miss. Catherine Helen "Carrie"   
889       1.0       1                              Behr, Mr. Karl Howell   
890       0.0       3                                Dooley, Mr. Patrick   

        Sex   Age  SibSp  Parch            Ticket     Fare Cabin Embarked  
0      male  22.0      1      0         A/5 21171   7.2500   NaN        S  
1    female  38.0      1      0          PC 17599  71.2833   C85        C  
2    female  26.0      0      0  STON/O2. 3101282   7.9250   NaN        S  
3    female  35.0      1      0            113803  53.1000  C123        S  
7      male   2.0      3      1            349909  21.0750   NaN        S  
..      ...   ...    ...    ...               ...      ...   ...      ...  
886    male  27.0      0      0            211536  13.0000   NaN        S  
887  female  19.0      0      0            112053  30.0000   B42        S  
888  female   NaN      1      2        W./C. 6607  23.4500   NaN        S  
889    male  26.0      0      0            111369  30.0000  C148        C  
890    male  32.0      0      0            370376   7.7500   NaN        Q  

[888 rows x 11 columns]

In [8]:
numerical_columns = df.select_dtypes(exclude='object').columns
categorical_columns = df.select_dtypes(include='object').columns

In [9]:
# basic_imputation(df, strategy_numerical='constant', strategy_categorical='constant', fill_value=123)

# basic_imputation(df, strategy_numerical='median', strategy_categorical='constant', fill_value='missing_value')

imputation(df, strategy_numerical='median', strategy_categorical='most_frequent',  numerical_columns = numerical_columns, categorical_columns = categorical_columns)

# basic_imputation(df, strategy_numerical='knn', strategy_categorical='most_frequent', n_neighbors=3, weights='uniform')

Survived  Pclass                                               Name  \
0         0.0     3.0                            Braund, Mr. Owen Harris   
1         1.0     1.0  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2         1.0     3.0                             Heikkinen, Miss. Laina   
3         1.0     1.0       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
7         0.0     3.0                     Palsson, Master. Gosta Leonard   
..        ...     ...                                                ...   
886       0.0     2.0                              Montvila, Rev. Juozas   
887       1.0     1.0                       Graham, Miss. Margaret Edith   
888       0.0     3.0           Johnston, Miss. Catherine Helen "Carrie"   
889       1.0     1.0                              Behr, Mr. Karl Howell   
890       0.0     3.0                                Dooley, Mr. Patrick   

        Sex   Age  SibSp  Parch            Ticket     Fare    Cabin Embarked  
0      male  22.0    1.0    0.0         A/5 21171   7.2500  B96 B98        S  
1    female  38.0    1.0    0.0          PC 17599  71.2833      C85        C  
2    female  26.0    0.0    0.0  STON/O2. 3101282   7.9250  B96 B98        S  
3    female  35.0    1.0    0.0            113803  53.1000     C123        S  
7      male   2.0    3.0    1.0            349909  21.0750  B96 B98        S  
..      ...   ...    ...    ...               ...      ...      ...      ...  
886    male  27.0    0.0    0.0            211536  13.0000  B96 B98        S  
887  female  19.0    0.0    0.0            112053  30.0000      B42        S  
888  female  28.0    1.0    2.0        W./C. 6607  23.4500  B96 B98        S  
889    male  26.0    0.0    0.0            111369  30.0000     C148        C  
890    male  32.0    0.0    0.0            370376   7.7500  B96 B98        Q  

[888 rows x 11 columns]

In [10]:
df.isnull().sum()

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Cabin       0
Embarked    0
dtype: int64

# Outlier

In [11]:
df.describe()

Survived      Pclass         Age       SibSp       Parch        Fare
count  888.000000  888.000000  888.000000  888.000000  888.000000  888.000000
mean     0.385135    2.308559   29.329020    0.524775    0.382883   32.236012
std      0.486901    0.835686   13.013933    1.104186    0.807113   49.760012
min      0.000000    1.000000    0.420000    0.000000    0.000000    0.000000
25%      0.000000    2.000000   22.000000    0.000000    0.000000    7.895800
50%      0.000000    3.000000   28.000000    0.000000    0.000000   14.454200
75%      1.000000    3.000000   35.000000    1.000000    0.000000   31.000000
max      1.000000    3.000000   80.000000    8.000000    6.000000  512.329200

In [12]:
df.shape

(888, 11)

In [13]:
df = inter_quartile_range(df, iqr_threshold = 1.5, Lc = 0.05, Uc = 0.95, cap = False, numerical_columns = numerical_columns)
df

Survived  Pclass                                               Name  \
0         0.0     3.0                            Braund, Mr. Owen Harris   
1         1.0     1.0  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2         1.0     3.0                             Heikkinen, Miss. Laina   
3         1.0     1.0       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
7         0.0     3.0                     Palsson, Master. Gosta Leonard   
..        ...     ...                                                ...   
886       0.0     2.0                              Montvila, Rev. Juozas   
887       1.0     1.0                       Graham, Miss. Margaret Edith   
888       0.0     3.0           Johnston, Miss. Catherine Helen "Carrie"   
889       1.0     1.0                              Behr, Mr. Karl Howell   
890       0.0     3.0                                Dooley, Mr. Patrick   

        Sex   Age  SibSp  Parch            Ticket     Fare    Cabin Embarked  
0      male  22.0    1.0    0.0         A/5 21171   7.2500  B96 B98        S  
1    female  38.0    1.0    0.0          PC 17599  71.2833      C85        C  
2    female  26.0    0.0    0.0  STON/O2. 3101282   7.9250  B96 B98        S  
3    female  35.0    1.0    0.0            113803  53.1000     C123        S  
7      male   2.0    3.0    1.0            349909  21.0750  B96 B98        S  
..      ...   ...    ...    ...               ...      ...      ...      ...  
886    male  27.0    0.0    0.0            211536  13.0000  B96 B98        S  
887  female  19.0    0.0    0.0            112053  30.0000      B42        S  
888  female  28.0    1.0    2.0        W./C. 6607  23.4500  B96 B98        S  
889    male  26.0    0.0    0.0            111369  30.0000     C148        C  
890    male  32.0    0.0    0.0            370376   7.7500  B96 B98        Q  

[877 rows x 11 columns]

In [14]:
df.shape

(877, 11)

# Transformation

In [15]:
df1 = df.copy()
df1.head()

Survived  Pclass                                               Name  \
0       0.0     3.0                            Braund, Mr. Owen Harris   
1       1.0     1.0  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2       1.0     3.0                             Heikkinen, Miss. Laina   
3       1.0     1.0       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
7       0.0     3.0                     Palsson, Master. Gosta Leonard   

      Sex   Age  SibSp  Parch            Ticket     Fare    Cabin Embarked  
0    male  22.0    1.0    0.0         A/5 21171   7.2500  B96 B98        S  
1  female  38.0    1.0    0.0          PC 17599  71.2833      C85        C  
2  female  26.0    0.0    0.0  STON/O2. 3101282   7.9250  B96 B98        S  
3  female  35.0    1.0    0.0            113803  53.1000     C123        S  
7    male   2.0    3.0    1.0            349909  21.0750  B96 B98        S

In [16]:
numerical_columns = df1.select_dtypes(exclude='object').columns
categorical_columns = df1.select_dtypes(include='object').columns

### Onehot Encodinng

In [17]:
# df1, encoder = onehot_transform(df1, categorical_columns = categorical_columns)
# df1.head()

### ordinal_transform

In [18]:
df1, encoder = ordinal_transform(df1, categorical_columns = categorical_columns)
df1.head()

Survived  Pclass   Name  Sex   Age  SibSp  Parch  Ticket     Fare  Cabin  \
0       0.0     3.0  107.0  1.0  22.0    1.0    0.0   520.0   7.2500   46.0   
1       1.0     1.0  188.0  0.0  38.0    1.0    0.0   592.0  71.2833   80.0   
2       1.0     3.0  350.0  0.0  26.0    0.0    0.0   664.0   7.9250   46.0   
3       1.0     1.0  270.0  0.0  35.0    1.0    0.0    49.0  53.1000   54.0   
7       0.0     3.0  618.0  1.0   2.0    3.0    1.0   393.0  21.0750   46.0   

   Embarked  
0       2.0  
1       0.0  
2       2.0  
3       2.0  
7       2.0

In [19]:
# df1[categorical_columns] = encoder.inverse_transform(df1[categorical_columns])
# df1.head()

### Label Encoding

In [20]:
# If the target column is categorical
df1, encoder = label_transform(df1, target_column='Survived')
df1.head()

Survived  Pclass   Name  Sex   Age  SibSp  Parch  Ticket     Fare  Cabin  \
0         0     3.0  107.0  1.0  22.0    1.0    0.0   520.0   7.2500   46.0   
1         1     1.0  188.0  0.0  38.0    1.0    0.0   592.0  71.2833   80.0   
2         1     3.0  350.0  0.0  26.0    0.0    0.0   664.0   7.9250   46.0   
3         1     1.0  270.0  0.0  35.0    1.0    0.0    49.0  53.1000   54.0   
7         0     3.0  618.0  1.0   2.0    3.0    1.0   393.0  21.0750   46.0   

   Embarked  
0       2.0  
1       0.0  
2       2.0  
3       2.0  
7       2.0

# Scaling

In [21]:
numerical_columns = df1.select_dtypes(exclude='object').columns
numerical_columns = numerical_columns.drop(['Survived'])
numerical_columns

Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Embarked'],
      dtype='object')

In [22]:
df1, scaler = standard_scale(df1, feature_columns=numerical_columns)
df1.head()

Survived    Pclass      Name       Sex       Age     SibSp     Parch  \
0         0  0.830930 -1.307433  0.737573 -0.558608  0.603257 -0.469245   
1         1 -1.566194 -0.987487 -1.355798  0.665151  0.603257 -0.469245   
2         1  0.830930 -0.347595 -1.355798 -0.252668 -0.527205 -0.469245   
3         1 -1.566194 -0.663591 -1.355798  0.435697  0.603257 -0.469245   
7         0  0.830930  0.710991  0.737573 -2.088307  2.864182  0.820810   

     Ticket      Fare     Cabin  Embarked  
0  0.936888 -0.558432 -0.285857  0.585623  
1  1.298963  0.994304  1.167208 -1.944383  
2  1.661039 -0.542064 -0.285857  0.585623  
3 -1.431691  0.553379  0.056041  0.585623  
7  0.298226 -0.223191 -0.285857  0.585623

# Feature selection

In [23]:
df2 = df1.copy()
print(df2.shape)
df2.head()

(877, 11)


Survived    Pclass      Name       Sex       Age     SibSp     Parch  \
0         0  0.830930 -1.307433  0.737573 -0.558608  0.603257 -0.469245   
1         1 -1.566194 -0.987487 -1.355798  0.665151  0.603257 -0.469245   
2         1  0.830930 -0.347595 -1.355798 -0.252668 -0.527205 -0.469245   
3         1 -1.566194 -0.663591 -1.355798  0.435697  0.603257 -0.469245   
7         0  0.830930  0.710991  0.737573 -2.088307  2.864182  0.820810   

     Ticket      Fare     Cabin  Embarked  
0  0.936888 -0.558432 -0.285857  0.585623  
1  1.298963  0.994304  1.167208 -1.944383  
2  1.661039 -0.542064 -0.285857  0.585623  
3 -1.431691  0.553379  0.056041  0.585623  
7  0.298226 -0.223191 -0.285857  0.585623

In [24]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

In [25]:
df2 = rfe(df2, label_column='Survived', n_features_to_select=7, estimator=GradientBoostingClassifier())
df2

Pclass      Name       Sex       Age    Ticket      Fare     Cabin  \
0    0.830930 -1.307433  0.737573 -0.558608  0.936888 -0.558432 -0.285857   
1   -1.566194 -0.987487 -1.355798  0.665151  1.298963  0.994304  1.167208   
2    0.830930 -0.347595 -1.355798 -0.252668  1.661039 -0.542064 -0.285857   
3   -1.566194 -0.663591 -1.355798  0.435697 -1.431691  0.553379  0.056041   
7    0.830930  0.710991  0.737573 -2.088307  0.298226 -0.223191 -0.285857   
..        ...       ...       ...       ...       ...       ...       ...   
886 -0.367632  0.414744  0.737573 -0.176183 -1.175221 -0.419001 -0.285857   
887 -1.566194 -0.545093 -1.355798 -0.788063 -1.607700 -0.006770 -1.012389   
888  0.830930 -0.110599 -1.355798 -0.099698  1.691212 -0.165600 -0.285857   
889 -1.566194 -1.414081  0.737573 -0.252668 -1.637873 -0.006770  0.269727   
890  0.830930 -0.868988  0.737573  0.206242  0.655273 -0.546307 -0.285857   

     Survived  
0           0  
1           1  
2           1  
3           1  
7           0  
..        ...  
886         0  
887         1  
888         0  
889         1  
890         0  

[877 rows x 8 columns]

# Model Building

In [40]:
from sklearn.utils import all_estimators

estimators = all_estimators(type_filter='regressor')

all_regs = []
for name, RegressorClass in estimators:
    try:
        print(name)
        # reg = RegressorClass()
        # all_regs.append(reg)
    except Exception as e:
        print(e)

ARDRegression
AdaBoostRegressor
BaggingRegressor
BayesianRidge
CCA
DecisionTreeRegressor
DummyRegressor
ElasticNet
ElasticNetCV
ExtraTreeRegressor
ExtraTreesRegressor
GammaRegressor
GaussianProcessRegressor
GradientBoostingRegressor
HistGradientBoostingRegressor
HuberRegressor
IsotonicRegression
KNeighborsRegressor
KernelRidge
Lars
LarsCV
Lasso
LassoCV
LassoLars
LassoLarsCV
LassoLarsIC
LinearRegression
LinearSVR
MLPRegressor
MultiOutputRegressor
MultiTaskElasticNet
MultiTaskElasticNetCV
MultiTaskLasso
MultiTaskLassoCV
NuSVR
OrthogonalMatchingPursuit
OrthogonalMatchingPursuitCV
PLSCanonical
PLSRegression
PassiveAggressiveRegressor
PoissonRegressor
QuantileRegressor
RANSACRegressor
RadiusNeighborsRegressor
RandomForestRegressor
RegressorChain
Ridge
RidgeCV
SGDRegressor
SVR
StackingRegressor
TheilSenRegressor
TransformedTargetRegressor
TweedieRegressor
VotingRegressor


In [27]:
df3 = df2.copy()
df3.shape

(877, 8)

In [28]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df3.drop('Survived', axis=1), df3['Survived'], 
                                                        test_size=0.15, random_state=42)

In [ ]:
model = SVR()

In [63]:
with mlflow.start_run(run_name=datetime.now().strftime('%Y-%m-%d_%H:%M:%S')) as run:
    run_id = run.info.run_id
    print('MLflow Run ID: {}'.format(run_id))
    client = MlflowClient()
    client.set_tag(run_id, "mlflow.note.content", f"ML model: {str(model)}")
    if "Xgboost" in str(model):
        mlflow.xgboost.autolog()
    else:
        mlflow.sklearn.autolog()
    
    tags = {"Application": "AutoPilotML"}
    mlflow.set_tags(tags)
    model = SVC()
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    # model = model.fit(x_train, y_train)
    # y_pred = model.predict(x_test)

2023/11/02 19:12:01 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/shyam/anaconda3/envs/autopilotml/lib/python3.10/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."


MLflow Run ID: b1aa9652e94a4dc4af3ecc6c3f4f86e3


2023/11/02 19:12:08 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/shyam/anaconda3/envs/autopilotml/lib/python3.10/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."


In [59]:
!mlflow ui

[2023-11-02 16:10:53 -0400] [44794] [INFO] Starting gunicorn 21.2.0
[2023-11-02 16:10:53 -0400] [44794] [INFO] Listening at: http://127.0.0.1:5000 (44794)
[2023-11-02 16:10:53 -0400] [44794] [INFO] Using worker: sync
[2023-11-02 16:10:53 -0400] [44795] [INFO] Booting worker with pid: 44795
[2023-11-02 16:10:53 -0400] [44796] [INFO] Booting worker with pid: 44796
[2023-11-02 16:10:53 -0400] [44797] [INFO] Booting worker with pid: 44797
[2023-11-02 16:10:53 -0400] [44798] [INFO] Booting worker with pid: 44798
^C
[2023-11-02 16:12:43 -0400] [44794] [INFO] Handling signal: int
[2023-11-02 16:12:43 -0400] [44797] [INFO] Worker exiting (pid: 44797)
[2023-11-02 16:12:43 -0400] [44796] [INFO] Worker exiting (pid: 44796)
[2023-11-02 16:12:43 -0400] [44798] [INFO] Worker exiting (pid: 44798)
[2023-11-02 16:12:43 -0400] [44795] [INFO] Worker exiting (pid: 44795)


In [96]:
categorical_columns = df.select_dtypes(include=['object']).columns
# categorical_columns = categorical_columns.drop(['Embarked'])
# categorical_columns = categorical_columns.drop(['Survived']) if 'Survived' in categorical_columns else categorical_columns
categorical_columns

Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')

In [92]:
drop_columns = [x for x in categorical_columns if df[x].nunique() > 0.3*100]
print(drop_columns)
df.drop(drop_columns, axis=1)

['Name', 'Ticket', 'Cabin']


PassengerId  Survived  Pclass     Sex   Age  SibSp  Parch     Fare  \
0              1       0.0       3    male  22.0      1      0   7.2500   
1              2       1.0       1  female  38.0      1      0  71.2833   
2              3       1.0       3  female  26.0      0      0   7.9250   
3              4       1.0       1  female  35.0      1      0  53.1000   
4              5       NaN       3    male  35.0      0      0   8.0500   
..           ...       ...     ...     ...   ...    ...    ...      ...   
886          887       0.0       2    male  27.0      0      0  13.0000   
887          888       1.0       1  female  19.0      0      0  30.0000   
888          889       0.0       3  female   NaN      1      2  23.4500   
889          890       1.0       1    male  26.0      0      0  30.0000   
890          891       0.0       3    male  32.0      0      0   7.7500   

    Embarked  
0          S  
1          C  
2          S  
3          S  
4          S  
..       ...  
886        S  
887        S  
888        S  
889        C  
890        Q  

[891 rows x 9 columns]

In [105]:
categorical_columns

Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')

In [109]:
label_column = 'PassengerId'

In [110]:
drop_columns = [x for x in df if df[x].nunique() > 0.3*100]
print(drop_columns)
[x for x in drop_columns if x not in label_column or x in categorical_columns]

['PassengerId', 'Name', 'Age', 'Ticket', 'Fare', 'Cabin']


['Name', 'Age', 'Ticket', 'Fare', 'Cabin']